In [35]:
import numpy as np
import pandas as pd
from scipy.stats import norm, chi2

# import the table, rename columns
col=['TC', 'Q', 'PL', 'PK', 'PF']
df=pd.read_table('Nerlove1963.txt', header=None, names=col)

# create a column for constant 1
df['one']=np.ones(df.shape[0])
# get the log
for co in col:
    df['log_'+co]=df.apply(lambda row: np.log(row[co]), axis=1)
cols=np.append(['log_'+co for co in col[1:]], 'one')

# calculate OLS estimator
X=df[cols]
y=df['log_TC']
beta=np.dot(
    np.linalg.inv(np.dot(X.T, X)), 
    np.dot(X.T, y))

# get residual and its square
def get_e_hat(row):
    return row['log_TC']-np.dot(row[cols].T, beta)
df['res']=df.apply(get_e_hat, axis=1)
df['e_hat_sqr']=df['res']**2

# calculate std_error by white
e_e=np.diag(df['e_hat_sqr'])
left=np.linalg.inv(np.dot(X.T, X))
mid=np.dot(np.dot(X.T, e_e), X)
right=left
White=np.dot(np.dot(left, mid), right)
std_err_white=[np.sqrt(White[i][i]) for i in range(X.shape[1])]

In [36]:
# a)
print('beta:',beta, '\n'
      'std_err:',std_err_white)

beta: [ 0.72039408  0.4363412  -0.21988835  0.42651695 -3.52650284] 
std_err: [0.032030570179413166, 0.24136354235687732, 0.31818018431970413, 0.074169868207054054, 1.6887096637897276]


In [30]:
#  d)
R=np.array([0,1,1,1,0]).T
left=np.dot(White,R)
mid=(np.dot(np.dot(R.T, White), R))**-1
right=np.dot(R.T, beta)-1
temp=np.dot(np.dot(left, mid), right)
beta_emd=beta-temp

# get residual and its square
def get_e_hat_new(row):
    return row['log_TC']-np.dot(np.transpose(row[cols]), beta_emd)
df['res_new']=df.apply(get_e_hat_new, axis=1)
df['e_hat_sqr_new']=df['res_new']**2

# calculate std_error by white
e_e=np.diag(df['e_hat_sqr_new'])
left_new=np.linalg.inv(np.dot(X.T, X))
mid_new=np.dot(np.dot(X.T, e_e), X)
right_new=left_new
White_new=np.dot(np.dot(left_new, mid_new), right_new)
std_err_white_new=[np.sqrt(White_new[i][i]) for i in range(X.shape[1])]

print('beta_emd:',beta_emd, '\n'
      'std_err_emd:',std_err_white_new)

beta_emd: [ 0.72019085  0.58051965  0.00921904  0.41026131 -4.74464602] 
std_err_emd: [0.031870909043272021, 0.24517983157769277, 0.31666165203255781, 0.074861333392265542, 1.6706552822372391]


In [34]:
# e)
# construct the Wald statistics for the joint hypo
R=np.array([0,1,1,1,0]).T
left=np.dot(R.T, beta)-1
mid=(np.dot(np.dot(R.T, White), R))**-1
right=left
Wald=np.dot(np.dot(left, mid), right)
# get critical value and 
c=chi2.ppf(0.95, df=np.linalg.matrix_rank(R))
print('Wald:', Wald, '\n',
      'crit_value', c, '\n',
['Fail to Reject','Reject'][Wald>c], '\n',
      'p-value:', 1-chi2.cdf(Wald, 1))

Wald: 0.66852632671 
 crit_value 3.84145882069 
 Fail to Reject 
 p-value: 0.413565868842
